# Wrangle OpenStreetMap Data
                  by George Liu, December 2015



### Overview
OpenStreetMap(OSM) is an open map database built by volunteers around the world. The data created are available for free use under the Open Database License (see Appendix for reference). As the data were crowdsourced, there can be inconsistency in how the data is presented. In this project, we will try to improve the data quality with data wrangling and store the cleaned data in MongoDB. In particular, XML map data will be parsed, audited and processed for street type inconsistenty before being reshaped and converted into JSON format data, later imported into the MongoDB database.  

Due to page limit, this report only include partial code for the project. The full wrangling script can be found on [GitHub](hhttps://github.com/georgeliu1998/Udacity_Nanodegree).



### Problems in the OSM data
In order to prepare the data for adding to the database, specifically, to check the "k" value for each tag and to make sure they can be valid keys in MongoDB, we use the "key_type" function in the script to examine and get the following result:

>{'lower': 2502530, 'lower_colon': 1760581, 'other': 129440, 'problemchars': 0}

With **"lower"** being tags that contain only lowercase letters and are valid, **"lower_colon"** for otherwise valid tags with a colon in their names, **"problemchars"** for tags with problematic characters, and **"other"** for other tags that do not fall into the other three categories.

After running the "audit_street_type" function, we are able to find a list of all street types that are not part of our standard unabbreviated street names. For example:
> 'Ave': {'455 Mayfair Ave', '480 Mayfair Ave', 'Champlain Ave', 'Raleigh Ave', 'Rawlinson Ave', 'Warden Ave'},

> 'Ave.': {'Dean Ave.', 'Islington Ave.', 'University Ave.'},

Not only street types can have different forms of abbreviations, such as "ave." versus "ave", but also misspelling is not uncommon, "Driver" instead of "Drive", to cite an example. Based on this, a mapping of problem names to preferred names is built. To fix all this, the "shape_elemen" function is run and the resulting data are stored into the final json file that is used to add data to the MongoDB database.

One other type of problems is attribute value inconsistency. For example, in some tags, the k attribute value is "lanes", and in others, the value is "lanes : forward".  This, in itself, may not necessary be a problem. However, when converting to json file, this can cause data structure issues. The problem I encountered was "TypeError" in Python when a dictionary was overwritten by the k value with same name. In the previous example, it would be a "lanes" dict being replaced as "lanes" string, which cannot be added dict items any more.

Some "tag" tags have "k" attribte of "type", as a result, this is in conflict with the predefined "type" key in the dict(as requested by the instruction of lesson 6 programming exercise). To resovle this, the one corresponding to the overall tag has been renamed to "tag_type". 

Finaly, there're instances where street name has multiple entries concatenated together, such as this one:
> 'name': '\\2/ Kingston Road, Port Union Road, Sheppard Avenue'

And the source for this document is:
> 'CanMatrix \xa9 Department of Natural Resources Canada. All rights reserved.', 

indicating there is some formatting issue in play that may need to be further cleaned in future project.

### Data Overview 
The data chosen for this project is the OSM Toronto dataset which is downloaded from the Mapzen website. Although the initial downloaded bz2 zip file was only **83,941 KB**, once it was decompressed, the size immediately jumped to **1,216,100 KB**. This new file is named "toronto_canada.osm" and has XML format.

Using the "count_tags" function in the script, the XML file is parsed and tag information is provided in the following output:

>defaultdict(int,
            {'bounds': 1,
             'member': 85911,
             'nd': 6085327,
             'node': 5452721,
             'osm': 1,
             'relation': 5713,
             'tag': 4392551,
             'way': 649048})

Then data was processed and saved into a new file "toronto_canada.osm.json". This new json format file has a size of **1,491,113 KB**. With this new file, the Toronto OSM data was then added into a MongoDB database named "osm", collection "to" using following code:

> mongoimport --db osm --collection to --file toronto_canada.osm.json

Further examination using the "count_unique" function gives us the total number of unique users: 1713. This is a little bit more than the result from the following MongoDB query for unique users as some data are removed in the cleaning process due to data integrity problems.

From here, we can use the MongoDB Python driver to further explore and summarize the data.

In [1]:
from pymongo import MongoClient
import pprint

client = MongoClient('localhost:27017')
db = client.osm
# Number of documents
db.to.find().count()

6101769

In [2]:
# Number of nodes
db.to.find({"tag_type":"node"}).count() 

5452721

In [3]:
# Number of ways
db.to.find({"tag_type":"way"}).count()

649048

In [4]:
# Number of nodes that are cafes
db.to.find({"tag_type" : "node", "amenity" : "cafe"}).count()

1296

In [5]:
# Number of unique users
len(db.to.distinct("created.user"))

1695

In [6]:
# Top Contributors
top10 = db.to.aggregate([{"$group" : {"_id" : "$created.user",
                                           "count" : {"$sum" : 1}}},
                              {"$sort" : {"count" : -1}},
                              {"$limit" : 10}])
pprint.pprint([doc for doc in top10][:3])

[{u'_id': u'andrewpmk', u'count': 4129928},
 {u'_id': u'MikeyCarter', u'count': 485239},
 {u'_id': u'Kevo', u'count': 391401}]


### Other ideas about the datasets
As mentioned previously, there are opportunities to further clean the data. One such example is to audit all the node names (name field of each document), and then try to fix invalid entries and/or standardize the entries. In order to do this, we need to better understand how the information for the fields in question is collected, and to have a "gold standard" to verify against. The benefit of doing this type of further cleaning is data quality improvement, which ensures the usefulness of the data - afterall, when data is not correct, queries will be problematic as well. However, more granular cleaning could also lead to more human error, so caution and proper procedures are recommended. 

In terms of overall process improvement to better the OSM data quality, one potential solution is to implement mandatory fields and automatic data verification at the time of editing. For example, when an OSM user tries to edit the name tag of a node, instead of allowing free entry, maybe some control mechnism can be in place so that it is not allowed to input more than one street names, i.e. with two or more street, avenue or similiar words showing. By doing this, the data quality can be greatly improved at the source. However, the level of efforts involved on the user's part can increase, and may lead to frustration, even less user contribution.

The dataset we produced in this project can be used in many ways, including contributing back to OSM to improve data quality, share with local government to better understand city business situation etc.

Finally, we run some more queries on the database to further explore this dataset.

In [7]:
# What is the percentage of ways that get updated this year?
updated_ways = db.to.find({"tag_type" : "way", 
                           "created.timestamp" : {"$regex" : "2013."}}).count()
all_ways = db.to.find({"tag_type":"way"}).count()
print("{0:.0f}%".format(float(updated_ways)/all_ways * 100))

13%


In [8]:
# Whic type of amenity is most numerous?
amenity = db.to.aggregate([
        {"$match" : {"amenity" : {"$exists" : 1}}},
        {"$group" : {"_id" : "$amenity",
                     "count" : {"$sum" : 1}}},
        {"$sort" : {"count" : -1}},
        {"$limit" : 1}
    ])
pprint.pprint([doc for doc in amenity])

[{u'_id': u'parking', u'count': 25585}]


In [9]:
# What are all the amenity types?
db.to.aggregate([
                 {"$group" : {"_id" : "amenity",
                              "amenity_set" : {"$addToSet" : "$amenity"}}
                              },
                 {"$sort" : {"_id" : -1}}
                ])

In [10]:
# How many restuurants are there on Yonge Stree?
db.to.find({"addr.street" : "Yonge Street", 
            "amenity" : "restaurant"}).count()

65

In [11]:
# Where can you find the most Chinese cuisine?
chinese_cuisine = db.to.aggregate([
        {"$match" : {"amenity" : "restaurant", 
                     "cuisine" : "chinese"}},
        {"$group" : {"_id" : "$address.street",
                     "count" : {"$sum" : 1}}
        },
        {"$sort" : {"count" : -1}},
        {"$limit" : 10}
    ])

In [12]:
# Which nodes get referenced most?
most_referred = db.to.aggregate([                     
                       {"$match" : {"node_refs" : {"$exists" : 1}}},
                       {"$unwind" : "$node_refs"},
                       {"$group" : {"_id" : "$node_refs", 
                                    "count" : {"$sum" : 1}}},
                       {"$sort" : {"count" : -1}},
                       {"$limit" : 1}
                     ], allowDiskUse = True)
pprint.pprint([n for n in most_referred])

[{u'_id': u'433821981', u'count': 10}]


### Appendix
1. [Full Code of the Project on GitHub](https://github.com/georgeliu1998/Udacity_Nanodegree)
2. [OSM Data Download Link](https://mapzen.com/data/metro-extracts)
3. [Link to the Map Position Wrangled(Toronto, Canada)](https://www.openstreetmap.org/relation/324211)
4. [OpenStreetMap Wikipedia Article](https://en.wikipedia.org/wiki/OpenStreetMap)
5. [About Page on OpenStreetMap Site](https://www.openstreetmap.org/about)
